# Tensorflow Bijectors simple example
Bijectors and shapes of the tensors which are produced by different functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tfd = tf.contrib.distributions
tfb = tfd.bijectors
layers = tf.contrib.layers

In [2]:
sess = tf.InteractiveSession()

# Comparing custom implementation of NF with implemented function

## Building simple bijector: $y = \sigma(x \cdot w + b)$

In [3]:
base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros([2], tf.float32))
bijectors = [
    tfb.AffineScalar(shift=tf.constant([0.2, 2.0]), scale=tf.constant([0.3, 1.2])),
    tfb.Sigmoid()
]

dist = tfd.TransformedDistribution(
    distribution=base_dist,
    bijector=tfb.Chain(list(reversed(bijectors))),
)

Manual computation of the log likelihood 

In [11]:
# some initial z
z = tf.constant([0.6, 0.5])

# inverse first bijector + accumulate log_det_jacobian
z_1 = bijectors[1].inverse(z)
log_z1_z = bijectors[1].inverse_log_det_jacobian(z)

# inverse second bijector + accumulate log_det_jacobian
z_0 = bijectors[0].inverse(z_1)
log_z0_z1 = bijectors[0].inverse_log_det_jacobian(z_1)

# final density
log_prob = base_dist.log_prob(z_0) + log_z1_z + log_z0_z1

In [12]:
dist.log_prob(z).eval()

array([-0.83020973, -2.2573261 ], dtype=float32)

In [13]:
log_prob.eval()

array([-0.83020973, -2.2573261 ], dtype=float32)

Compare this with implemented function

In [14]:
dist.prob(z).eval()

array([0.43595785, 0.10462989], dtype=float32)

In [16]:
base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros([2, 4], tf.float32))
base_dist.sample()        # [2, 4]
base_dist.sample(3)       # [3, 2, 4]
base_dist.sample([3, 5])  # [3, 5, 2, 4]

<tf.Tensor 'MultivariateNormalDiag_23/sample/affine_linear_operator/forward/add:0' shape=(3, 5, 2, 4) dtype=float32>

In [15]:
base_dist.sample([3, 5])

<tf.Tensor 'MultivariateNormalDiag_17/sample/affine_linear_operator/forward/add:0' shape=(3, 5, 2, 4) dtype=float32>

# Shape of samples from different distributions

In [21]:
dist = tfd.TransformedDistribution(
    distribution=base_dist,
    bijector=tfb.Sigmoid(),
)

dist.sample()       # [2, 4]
dist.sample(3)      # [3, 2, 4]
dist.sample([3, 5]) # [3, 5, 2, 4]

<tf.Tensor 'sigmoidMultivariateNormalDiag_10/sample/sigmoid/forward/Sigmoid:0' shape=(3, 5, 2, 4) dtype=float32>

In [40]:
base_dist = tfd.Normal(loc=0.0, scale=1.0) # must be scalar
dist = tfd.TransformedDistribution(
    batch_shape=[2], 
    event_shape=[4],
    distribution=base_dist,
    bijector=tfb.Sigmoid(),
)

dist.sample()       # [2, 4]
dist.sample(3)      # [3, 2, 4]
dist.sample([3, 5]) # [3, 5, 2, 4]

<tf.Tensor 'sigmoidNormal_17/sample/sigmoid/forward/Sigmoid:0' shape=(3, 5, 2, 4) dtype=float32>

In [44]:
sigmoid_bijector = tfb.Exp()
x = tf.random_normal(shape=[3, 4])
sigmoid_bijector.forward(x) # [3, 4]
sigmoid_bijector.forward_log_det_jacobian(x) # [3, 4]

<tf.Tensor 'exp_6/forward_log_det_jacobian/Neg_1:0' shape=(3, 4) dtype=float32>

In [47]:
sigmoid_bijector = tfb.Exp(event_ndims=1)
x = tf.random_normal(shape=[3, 4])
sigmoid_bijector.forward(x) # [3, 4]
sigmoid_bijector.forward_log_det_jacobian(x) # [3]

<tf.Tensor 'exp_13/forward_log_det_jacobian/Neg_1:0' shape=(3,) dtype=float32>

In [49]:
sigmoid_bijector = tfb.Exp(event_ndims=2)
x = tf.random_normal(shape=[3, 4])
sigmoid_bijector.forward(x) # [3, 4]
sigmoid_bijector.forward_log_det_jacobian(x) # []

<tf.Tensor 'exp_18/forward/Exp:0' shape=(3, 4) dtype=float32>